In [1]:
import tensorflow as tf
import numpy as np
from msalib import layers
from msalib import network
from msalib import train
from msalib.utils import Dataset
import yaml


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
def target(x):
    """Target function

    Arguments:
        x {numpy array} -- input

    Returns:
        numpy array -- output
    """

    return np.sin(x)


def loss_func(output, label):
    """Loss function (l2)

    Arguments:
        output {tf tensor} -- output from network
        label {tf tensor} -- labels

    Returns:
        tf tensor -- loss
    """

    output = tf.reduce_sum(output, axis=1, keepdims=True)
    return tf.losses.mean_squared_error(
        predictions=output, labels=label)


def generate_datasets(config):
    """Generate synthetic dataset

    Arguments:
        config {dict} -- read from config.yml

    Returns:
        tuple of Dataset objects -- train/test sets
    """

    np.random.seed(config['seed'])
    num_train, num_test = config['num_train'], config['num_test']
    x_train = np.random.uniform(-np.pi, np.pi, (num_train, 1))
    x_test = np.random.uniform(-np.pi, np.pi, (num_test, 1))
    y_train, y_test = target(x_train), target(x_test)
    x_train = np.repeat(x_train, axis=1, repeats=config['num_nodes'])
    x_test = np.repeat(x_test, axis=1, repeats=config['num_nodes'])
    trainset = Dataset(data=x_train, target=y_train)
    testset = Dataset(data=x_test, target=y_test)
    return trainset, testset


def run_toy(config):
    """Run toy problem

    Arguments:
        config {dict} -- read from config.yml
    """

    print('='*100)
    print('Run toy model')
    print('='*100)
    print('Config:')
    for key, value in config.items():
        print('{:20} ({})'.format(key, value))
    print('='*100)

    tf.logging.set_verbosity(
        getattr(tf.logging, config['verbosity']))
    tf.set_random_seed(config['seed'])

    # Generate data
    trainset, testset = generate_datasets(config)

    # Build network
    input = tf.placeholder(
        tf.float32, [None, config['num_nodes']], name='input')
    output = tf.placeholder(tf.float32, [None, 1], name='output')
    net = network.Network(name='msa_net')

    for n in range(config['num_layers']):
        if n == 0:
            net.add(layers.ResidualDense(
                input_shape=(input.shape[1:]),
                units=config['num_nodes'], activation=config['activation'],
                msa_rho=config['rho'], msa_reg=config['reg'],
                kernel_initializer=tf.truncated_normal_initializer(
                    stddev=config['kernel_init']),
                bias_initializer=tf.constant_initializer(config['bias_init']),
                delta=config['delta'], name='residualdense_{}'.format(n)))
        else:
            net.add(layers.ResidualDense(
                units=config['num_nodes'], activation=config['activation'],
                msa_rho=config['rho'], msa_reg=config['reg'],
                kernel_initializer=tf.truncated_normal_initializer(
                    stddev=config['kernel_init']),
                bias_initializer=tf.constant_initializer(config['bias_init']),
                delta=config['delta'], name='residualdense_{}'.format(n)))

    net.msa_compute_x(input)
    net.msa_compute_p(output, loss_func)
    net.summary()
    sess = tf.Session()

    # MSA trainer
    msa_trainer = train.MSATrainer(
        network=net,
        name='MSA_trainer',
        maxiter=config['msa_maxiter'],
        perturb_init=config['msa_perturb_init'])
    msa_trainer.initialize(sess)
    msa_trainer.train(
        session=sess,
        trainset=trainset,
        testset=testset,
        batch_size=config['batch_size'],
        num_epochs=config['num_epochs'],
        buffer_size=config['buffer_size'],
        print_step=config['print_step'])

    # SGD trainer
    sgd_trainer = train.BPTrainer(
        network=net, name='SGD_trainer',
        method='GradientDescentOptimizer',
        args={'learning_rate': config['sgd_lr']})
    sgd_trainer.initialize(sess)
    sgd_trainer.train(
        session=sess,
        trainset=trainset,
        testset=testset,
        batch_size=config['batch_size'],
        num_epochs=config['num_epochs'],
        buffer_size=config['buffer_size'],
        print_step=config['print_step'])

    # Adagrad trainer
    adagrad_trainer = train.BPTrainer(
        network=net, name='Adagrad_trainer',
        method='AdagradOptimizer',
        args={'learning_rate': config['adagrad_lr']})
    adagrad_trainer.initialize(sess)
    adagrad_trainer.train(
        session=sess,
        trainset=trainset,
        testset=testset,
        batch_size=config['batch_size'],
        num_epochs=config['num_epochs'],
        buffer_size=config['buffer_size'],
        print_step=config['print_step'])

    # Adam trainer
    Adam_trainer = train.BPTrainer(
        network=net, name='Adam_trainer',
        method='AdamOptimizer',
        args={'learning_rate': config['adam_lr']})
    Adam_trainer.initialize(sess)
    Adam_trainer.train(
        session=sess,
        trainset=trainset,
        testset=testset,
        batch_size=config['batch_size'],
        num_epochs=config['num_epochs'],
        buffer_size=config['buffer_size'],
        print_step=config['print_step'])

In [4]:
!python3 -u main_toy.py | tee test.log


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Run toy model
Config:
seed                 (1)
verbosity            (WARN)
print_step           (False)
num_train            (100)
num_test             (100)
num_nodes            (5)
num_layers           (20)
rho                  (1.0)
reg                  (0.001)
activation           (tanh)
kernel_init          (0.0)
bias_init            (0.0)
delta                (0.25)
batch_size           (100)
buffer_size          (100)
num_epochs           (500)
sgd_lr               (0.1)
adagrad_lr           (0.1)
adam_lr              (0.005)
msa_maxiter          (10)
msa_perturb_init     (0.005)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
__

Epoch: 33
Train loss: 0.22351399064064026
Test loss: 0.24604740738868713
Epoch: 34
Train loss: 0.20563723146915436
Test loss: 0.22337844967842102
Epoch: 35
Train loss: 0.18954342603683472
Test loss: 0.20331339538097382
Epoch: 36
Train loss: 0.17644044756889343
Test loss: 0.18731528520584106
Epoch: 37
Train loss: 0.16589754819869995
Test loss: 0.1746366322040558
Epoch: 38
Train loss: 0.1572844535112381
Test loss: 0.1645336151123047
Epoch: 39
Train loss: 0.14921581745147705
Test loss: 0.15505564212799072
Epoch: 40
Train loss: 0.14235563576221466
Test loss: 0.14711681008338928
Epoch: 41
Train loss: 0.13658642768859863
Test loss: 0.1405756026506424
Epoch: 42
Train loss: 0.13177496194839478
Test loss: 0.13509342074394226
Epoch: 43
Train loss: 0.12678222358226776
Test loss: 0.12960253655910492
Epoch: 44
Train loss: 0.12229878455400467
Test loss: 0.1247115433216095
Epoch: 45
Train loss: 0.11878462880849838
Test loss: 0.12081722170114517
Epoch: 46
Train loss: 0.11566396057605743
Test loss: 0.1

Epoch: 145
Train loss: 0.04523574188351631
Test loss: 0.04564216732978821
Epoch: 146
Train loss: 0.04505039379000664
Test loss: 0.04547454044222832
Epoch: 147
Train loss: 0.0448489710688591
Test loss: 0.045253027230501175
Epoch: 148
Train loss: 0.04467804729938507
Test loss: 0.04512645676732063
Epoch: 149
Train loss: 0.04447823017835617
Test loss: 0.04487762600183487
Epoch: 150
Train loss: 0.04430907964706421
Test loss: 0.044735390692949295
Epoch: 151
Train loss: 0.04416695237159729
Test loss: 0.044608648866415024
Epoch: 152
Train loss: 0.0439971461892128
Test loss: 0.044451985508203506
Epoch: 153
Train loss: 0.043827857822179794
Test loss: 0.04427807778120041
Epoch: 154
Train loss: 0.04368826746940613
Test loss: 0.04413224756717682
Epoch: 155
Train loss: 0.04351846128702164
Test loss: 0.043954573571681976
Epoch: 156
Train loss: 0.04334717243909836
Test loss: 0.04377443343400955
Epoch: 157
Train loss: 0.04318348318338394
Test loss: 0.043638937175273895
Epoch: 158
Train loss: 0.04304970

Epoch: 255
Train loss: 0.033106729388237
Test loss: 0.033615000545978546
Epoch: 256
Train loss: 0.03304172679781914
Test loss: 0.03356116637587547
Epoch: 257
Train loss: 0.0329633504152298
Test loss: 0.033475134521722794
Epoch: 258
Train loss: 0.03287122771143913
Test loss: 0.03338172286748886
Epoch: 259
Train loss: 0.032819122076034546
Test loss: 0.03333952650427818
Epoch: 260
Train loss: 0.03275906667113304
Test loss: 0.03329060226678848
Epoch: 261
Train loss: 0.03269891440868378
Test loss: 0.033222079277038574
Epoch: 262
Train loss: 0.03264357149600983
Test loss: 0.033164460211992264
Epoch: 263
Train loss: 0.03259233012795448
Test loss: 0.03314798325300217
Epoch: 264
Train loss: 0.0325276181101799
Test loss: 0.033071041107177734
Epoch: 265
Train loss: 0.03246038779616356
Test loss: 0.032988931983709335
Epoch: 266
Train loss: 0.0323958545923233
Test loss: 0.03293957561254501
Epoch: 267
Train loss: 0.032345954328775406
Test loss: 0.0328943207859993
Epoch: 268
Train loss: 0.03228916227

Epoch: 365
Train loss: 0.027948785573244095
Test loss: 0.028531523421406746
Epoch: 366
Train loss: 0.027924777939915657
Test loss: 0.028499312698841095
Epoch: 367
Train loss: 0.027886344119906425
Test loss: 0.028483115136623383
Epoch: 368
Train loss: 0.027843404561281204
Test loss: 0.028432399034500122
Epoch: 369
Train loss: 0.02780495025217533
Test loss: 0.02841034159064293
Epoch: 370
Train loss: 0.027755770832300186
Test loss: 0.028339965268969536
Epoch: 371
Train loss: 0.027736425399780273
Test loss: 0.02833525836467743
Epoch: 372
Train loss: 0.027700847014784813
Test loss: 0.02828812412917614
Epoch: 373
Train loss: 0.027664199471473694
Test loss: 0.028258653357625008
Epoch: 374
Train loss: 0.027630090713500977
Test loss: 0.02821090817451477
Epoch: 375
Train loss: 0.027592435479164124
Test loss: 0.02818564511835575
Epoch: 376
Train loss: 0.027568314224481583
Test loss: 0.028158601373434067
Epoch: 377
Train loss: 0.027524227276444435
Test loss: 0.028110669925808907
Epoch: 378
Train l

...

Test loss: 0.28672921657562256
Epoch: 416
Train loss: 0.26512712240219116
Test loss: 0.28670135140419006
Epoch: 417
Train loss: 0.26510751247406006
Test loss: 0.2866736352443695
Epoch: 418
Train loss: 0.26508787274360657
Test loss: 0.28664594888687134
Epoch: 419
Train loss: 0.26506826281547546
Test loss: 0.2866183817386627
Epoch: 420
Train loss: 0.26504868268966675
Test loss: 0.28659093379974365
Epoch: 421
Train loss: 0.2650292217731476
Test loss: 0.28656360507011414
Epoch: 422
Train loss: 0.26500970125198364
Test loss: 0.2865363657474518
Epoch: 423
Train loss: 0.26499029994010925
Test loss: 0.286509245634079
Epoch: 424
Train loss: 0.2649708688259125
Test loss: 0.28648218512535095
Epoch: 425
Train loss: 0.2649514675140381
Test loss: 0.2864552140235901
Epoch: 426
Train loss: 0.26493215560913086
Test loss: 0.28642839193344116
Epoch: 427
Train loss: 0.26491284370422363
Test loss: 0.286401629447937
Epoch: 428
Train loss: 0.2648935914039612
Test loss: 0.2863749861717224
Epoch: 429
Train los

Train loss: 0.8077226281166077
Test loss: 1.0298640727996826
Epoch: 34
Train loss: 0.7807613611221313
Test loss: 0.9949867725372314
Epoch: 35
Train loss: 0.7564787268638611
Test loss: 0.9634369611740112
Epoch: 36
Train loss: 0.7345443367958069
Test loss: 0.9348326325416565
Epoch: 37
Train loss: 0.7146674990653992
Test loss: 0.9088375568389893
Epoch: 38
Train loss: 0.6965916752815247
Test loss: 0.8851546049118042
Epoch: 39
Train loss: 0.6800919771194458
Test loss: 0.8635215163230896
Epoch: 40
Train loss: 0.6649719476699829
Test loss: 0.8437072038650513
Epoch: 41
Train loss: 0.6510618925094604
Test loss: 0.8255088329315186
Epoch: 42
Train loss: 0.6382154226303101
Test loss: 0.8087488412857056
Epoch: 43
Train loss: 0.6263086795806885
Test loss: 0.7932726144790649
Epoch: 44
Train loss: 0.6152364611625671
Test loss: 0.778945803642273
Epoch: 45
Train loss: 0.6049107313156128
Test loss: 0.7656511068344116
Epoch: 46
Train loss: 0.5952566862106323
Test loss: 0.7532860040664673
Epoch: 47
Train l

Test loss: 0.40341290831565857
Epoch: 151
Train loss: 0.33304861187934875
Test loss: 0.4021748900413513
Epoch: 152
Train loss: 0.332214891910553
Test loss: 0.4009527564048767
Epoch: 153
Train loss: 0.33139288425445557
Test loss: 0.3997461199760437
Epoch: 154
Train loss: 0.3305823504924774
Test loss: 0.3985546827316284
Epoch: 155
Train loss: 0.32978326082229614
Test loss: 0.39737823605537415
Epoch: 156
Train loss: 0.32899534702301025
Test loss: 0.39621639251708984
Epoch: 157
Train loss: 0.3282184302806854
Test loss: 0.3950689435005188
Epoch: 158
Train loss: 0.32745233178138733
Test loss: 0.39393559098243713
Epoch: 159
Train loss: 0.3266969621181488
Test loss: 0.39281612634658813
Epoch: 160
Train loss: 0.32595202326774597
Test loss: 0.3917105197906494
Epoch: 161
Train loss: 0.3252173662185669
Test loss: 0.39061832427978516
Epoch: 162
Train loss: 0.32449302077293396
Test loss: 0.38953959941864014
Epoch: 163
Train loss: 0.3237786591053009
Test loss: 0.3884740173816681
Epoch: 164
Train loss

Test loss: 0.32551753520965576
Epoch: 266
Train loss: 0.2838577926158905
Test loss: 0.32518184185028076
Epoch: 267
Train loss: 0.28366801142692566
Test loss: 0.3248496651649475
Epoch: 268
Train loss: 0.28348034620285034
Test loss: 0.3245205879211426
Epoch: 269
Train loss: 0.28329482674598694
Test loss: 0.3241948187351227
Epoch: 270
Train loss: 0.28311148285865784
Test loss: 0.32387226819992065
Epoch: 271
Train loss: 0.2829301953315735
Test loss: 0.32355284690856934
Epoch: 272
Train loss: 0.28275102376937866
Test loss: 0.3232366144657135
Epoch: 273
Train loss: 0.2825739085674286
Test loss: 0.322923481464386
Epoch: 274
Train loss: 0.2823987603187561
Test loss: 0.322613388299942
Epoch: 275
Train loss: 0.28222566843032837
Test loss: 0.3223062753677368
Epoch: 276
Train loss: 0.2820545434951782
Test loss: 0.3220021724700928
Epoch: 277
Train loss: 0.2818852961063385
Test loss: 0.3217010200023651
Epoch: 278
Train loss: 0.28171804547309875
Test loss: 0.32140278816223145
Epoch: 279
Train loss: 0

Test loss: 0.30036303400993347
Epoch: 392
Train loss: 0.270714670419693
Test loss: 0.3002510070800781
Epoch: 393
Train loss: 0.2706619203090668
Test loss: 0.30013975501060486
Epoch: 394
Train loss: 0.270609587430954
Test loss: 0.30002930760383606
Epoch: 395
Train loss: 0.2705577611923218
Test loss: 0.2999197840690613
Epoch: 396
Train loss: 0.2705063223838806
Test loss: 0.29981106519699097
Epoch: 397
Train loss: 0.2704553008079529
Test loss: 0.29970309138298035
Epoch: 398
Train loss: 0.2704046964645386
Test loss: 0.2995959222316742
Epoch: 399
Train loss: 0.2703545391559601
Test loss: 0.2994895577430725
Epoch: 400
Train loss: 0.2703048288822174
Test loss: 0.2993839681148529
Epoch: 401
Train loss: 0.27025550603866577
Test loss: 0.29927918314933777
Epoch: 402
Train loss: 0.2702065408229828
Test loss: 0.29917511343955994
Epoch: 403
Train loss: 0.2701580226421356
Test loss: 0.29907187819480896
Epoch: 404
Train loss: 0.2701098620891571
Test loss: 0.2989693582057953
Epoch: 405
Train loss: 0.27

In [3]:
# if __name__ == "__main__":
#     config = yaml.safe_load(open("config.yml"))
#     run_toy(config['toy'])
    

Run toy model
Config:
seed                 (1)
verbosity            (WARN)
print_step           (False)
num_train            (100)
num_test             (100)
num_nodes            (5)
num_layers           (20)
rho                  (1.0)
reg                  (0.001)
activation           (tanh)
kernel_init          (0.0)
bias_init            (0.0)
delta                (0.25)
batch_size           (100)
buffer_size          (100)
num_epochs           (500)
sgd_lr               (0.1)
adagrad_lr           (0.1)
adam_lr              (0.005)
msa_maxiter          (10)
msa_perturb_init     (0.005)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
residualdense_0 (ResidualDen (None, 5)                 30        
________________________________________

Epoch: 45
Train loss: 0.11878462880849838
Test loss: 0.12081722170114517
Epoch: 46
Train loss: 0.11566396057605743
Test loss: 0.11742264032363892
Epoch: 47
Train loss: 0.1127704381942749
Test loss: 0.11432130634784698
Epoch: 48
Train loss: 0.1096314862370491
Test loss: 0.11101696640253067
Epoch: 49
Train loss: 0.10689201951026917
Test loss: 0.10819574445486069
Epoch: 50
Train loss: 0.10355933010578156
Test loss: 0.10460850596427917
Epoch: 51
Train loss: 0.101222924888134
Test loss: 0.10218772292137146
Epoch: 52
Train loss: 0.099486343562603
Test loss: 0.1003652811050415
Epoch: 53
Train loss: 0.09693531692028046
Test loss: 0.097777359187603
Epoch: 54
Train loss: 0.09542710334062576
Test loss: 0.09622682631015778
Epoch: 55
Train loss: 0.09394341707229614
Test loss: 0.09467333555221558
Epoch: 56
Train loss: 0.0917004719376564
Test loss: 0.09243670105934143
Epoch: 57
Train loss: 0.09005194157361984
Test loss: 0.09080243110656738
Epoch: 58
Train loss: 0.08838255703449249
Test loss: 0.089161

Epoch: 157
Train loss: 0.04318348318338394
Test loss: 0.043638937175273895
Epoch: 158
Train loss: 0.043049704283475876
Test loss: 0.04349137842655182
Epoch: 159
Train loss: 0.04289362207055092
Test loss: 0.04331818222999573
Epoch: 160
Train loss: 0.04272432252764702
Test loss: 0.043149787932634354
Epoch: 161
Train loss: 0.04255576431751251
Test loss: 0.04297732561826706
Epoch: 162
Train loss: 0.042432110756635666
Test loss: 0.042862337082624435
Epoch: 163
Train loss: 0.04227207973599434
Test loss: 0.04270379990339279
Epoch: 164
Train loss: 0.04211978614330292
Test loss: 0.042541906237602234
Epoch: 165
Train loss: 0.04193400591611862
Test loss: 0.04235941544175148
Epoch: 166
Train loss: 0.04178958386182785
Test loss: 0.042221490293741226
Epoch: 167
Train loss: 0.04166826605796814
Test loss: 0.042111415416002274
Epoch: 168
Train loss: 0.04154011234641075
Test loss: 0.04198412224650383
Epoch: 169
Train loss: 0.04139349237084389
Test loss: 0.041829731315374374
Epoch: 170
Train loss: 0.0412

Epoch: 267
Train loss: 0.032345954328775406
Test loss: 0.0328943207859993
Epoch: 268
Train loss: 0.032289162278175354
Test loss: 0.03282513841986656
Epoch: 269
Train loss: 0.03223578631877899
Test loss: 0.03277904540300369
Epoch: 270
Train loss: 0.03217713162302971
Test loss: 0.03272049501538277
Epoch: 271
Train loss: 0.03212563320994377
Test loss: 0.03266539424657822
Epoch: 272
Train loss: 0.03209228813648224
Test loss: 0.03262634947896004
Epoch: 273
Train loss: 0.03205612301826477
Test loss: 0.03259371966123581
Epoch: 274
Train loss: 0.03200172632932663
Test loss: 0.03252768889069557
Epoch: 275
Train loss: 0.031924229115247726
Test loss: 0.03245112672448158
Epoch: 276
Train loss: 0.03187728300690651
Test loss: 0.032411057502031326
Epoch: 277
Train loss: 0.03182768449187279
Test loss: 0.03235883638262749
Epoch: 278
Train loss: 0.03177323937416077
Test loss: 0.03230869024991989
Epoch: 279
Train loss: 0.031732309609651566
Test loss: 0.03225790336728096
Epoch: 280
Train loss: 0.031677111

Epoch: 377
Train loss: 0.027524227276444435
Test loss: 0.028110669925808907
Epoch: 378
Train loss: 0.027484171092510223
Test loss: 0.028072591871023178
Epoch: 379
Train loss: 0.02745509333908558
Test loss: 0.028025474399328232
Epoch: 380
Train loss: 0.027423713356256485
Test loss: 0.02800414338707924
Epoch: 381
Train loss: 0.027393681928515434
Test loss: 0.02799397148191929
Epoch: 382
Train loss: 0.027351222932338715
Test loss: 0.027928084135055542
Epoch: 383
Train loss: 0.027322599664330482
Test loss: 0.027896001935005188
Epoch: 384
Train loss: 0.027288563549518585
Test loss: 0.027845710515975952
Epoch: 385
Train loss: 0.027273930609226227
Test loss: 0.02785268984735012
Epoch: 386
Train loss: 0.02724010869860649
Test loss: 0.027829254046082497
Epoch: 387
Train loss: 0.027213364839553833
Test loss: 0.027789950370788574
Epoch: 388
Train loss: 0.027189111337065697
Test loss: 0.027790963649749756
Epoch: 389
Train loss: 0.02715689316391945
Test loss: 0.027751363813877106
Epoch: 390
Train l

...

Epoch: 345
Train loss: 0.26663658022880554
Test loss: 0.288997620344162
Epoch: 346
Train loss: 0.2666134834289551
Test loss: 0.28895989060401917
Epoch: 347
Train loss: 0.266590416431427
Test loss: 0.28892233967781067
Epoch: 348
Train loss: 0.2665673792362213
Test loss: 0.28888487815856934
Epoch: 349
Train loss: 0.2665444612503052
Test loss: 0.2888476550579071
Epoch: 350
Train loss: 0.2665216028690338
Test loss: 0.288810670375824
Epoch: 351
Train loss: 0.26649874448776245
Test loss: 0.2887738347053528
Epoch: 352
Train loss: 0.26647600531578064
Test loss: 0.2887371778488159
Epoch: 353
Train loss: 0.2664533257484436
Test loss: 0.2887006998062134
Epoch: 354
Train loss: 0.26643064618110657
Test loss: 0.28866440057754517
Epoch: 355
Train loss: 0.26640811562538147
Test loss: 0.2886282503604889
Epoch: 356
Train loss: 0.26638561487197876
Test loss: 0.28859224915504456
Epoch: 357
Train loss: 0.2663631737232208
Test loss: 0.2885564863681793
Epoch: 358
Train loss: 0.26634079217910767
Test loss: 0.

Train loss: 0.26362234354019165
Test loss: 0.28471434116363525
Epoch: 499
Train loss: 0.2636052370071411
Test loss: 0.2846930921077728
Trainer: BPTrainer (Adam_trainer)
Settings: {'method': 'AdamOptimizer', 'args': {'learning_rate': 0.005}}
Epoch: init
Train loss: 76.24555969238281
Test loss: 89.11460876464844
Epoch: 0
Train loss: 58.344974517822266
Test loss: 68.31175231933594
Epoch: 1
Train loss: 44.74446487426758
Test loss: 52.48972702026367
Epoch: 2
Train loss: 34.507015228271484
Test loss: 40.5676383972168
Epoch: 3
Train loss: 26.82599449157715
Test loss: 31.612756729125977
Epoch: 4
Train loss: 21.054109573364258
Test loss: 24.87596321105957
Epoch: 5
Train loss: 16.695707321166992
Test loss: 19.783010482788086
Epoch: 6
Train loss: 13.382044792175293
Test loss: 15.906180381774902
Epoch: 7
Train loss: 10.842714309692383
Test loss: 12.93144416809082
Epoch: 8
Train loss: 8.88032341003418
Test loss: 10.629358291625977
Epoch: 9
Train loss: 7.350700378417969
Test loss: 8.832228660583496


Epoch: 139
Train loss: 0.34403982758522034
Test loss: 0.41817936301231384
Epoch: 140
Train loss: 0.3430497944355011
Test loss: 0.4167441129684448
Epoch: 141
Train loss: 0.3420740067958832
Test loss: 0.4153282046318054
Epoch: 142
Train loss: 0.3411121666431427
Test loss: 0.4139314889907837
Epoch: 143
Train loss: 0.34016403555870056
Test loss: 0.41255369782447815
Epoch: 144
Train loss: 0.33922940492630005
Test loss: 0.41119450330734253
Epoch: 145
Train loss: 0.3383081555366516
Test loss: 0.4098537266254425
Epoch: 146
Train loss: 0.33740007877349854
Test loss: 0.40853098034858704
Epoch: 147
Train loss: 0.3365047872066498
Test loss: 0.4072258472442627
Epoch: 148
Train loss: 0.3356221318244934
Test loss: 0.4059380292892456
Epoch: 149
Train loss: 0.3347521126270294
Test loss: 0.4046672582626343
Epoch: 150
Train loss: 0.3338943421840668
Test loss: 0.40341290831565857
Epoch: 151
Train loss: 0.33304861187934875
Test loss: 0.4021748900413513
Epoch: 152
Train loss: 0.332214891910553
Test loss: 0.

Epoch: 296
Train loss: 0.27900761365890503
Test loss: 0.3164956569671631
Epoch: 297
Train loss: 0.2788723409175873
Test loss: 0.31624671816825867
Epoch: 298
Train loss: 0.2787385582923889
Test loss: 0.3160001039505005
Epoch: 299
Train loss: 0.2786061763763428
Test loss: 0.31575578451156616
Epoch: 300
Train loss: 0.27847525477409363
Test loss: 0.3155137002468109
Epoch: 301
Train loss: 0.27834582328796387
Test loss: 0.31527388095855713
Epoch: 302
Train loss: 0.27821770310401917
Test loss: 0.31503620743751526
Epoch: 303
Train loss: 0.27809104323387146
Test loss: 0.3148007392883301
Epoch: 304
Train loss: 0.2779656946659088
Test loss: 0.3145674467086792
Epoch: 305
Train loss: 0.277841717004776
Test loss: 0.3143361806869507
Epoch: 306
Train loss: 0.27771902084350586
Test loss: 0.31410712003707886
Epoch: 307
Train loss: 0.27759769558906555
Test loss: 0.31388014554977417
Epoch: 308
Train loss: 0.2774776518344879
Test loss: 0.31365519762039185
Epoch: 309
Train loss: 0.27735891938209534
Test los

Train loss: 0.26860329508781433
Test loss: 0.2957238256931305
Epoch: 441
Train loss: 0.26856690645217896
Test loss: 0.2956446409225464
Epoch: 442
Train loss: 0.2685306966304779
Test loss: 0.29556605219841003
Epoch: 443
Train loss: 0.26849472522735596
Test loss: 0.2954878807067871
Epoch: 444
Train loss: 0.2684589922428131
Test loss: 0.29541027545928955
Epoch: 445
Train loss: 0.26842355728149414
Test loss: 0.2953331768512726
Epoch: 446
Train loss: 0.2683883309364319
Test loss: 0.2952565550804138
Epoch: 447
Train loss: 0.26835331320762634
Test loss: 0.29518043994903564
Epoch: 448
Train loss: 0.2683185636997223
Test loss: 0.29510483145713806
Epoch: 449
Train loss: 0.26828405261039734
Test loss: 0.29502972960472107
Epoch: 450
Train loss: 0.2682497203350067
Test loss: 0.2949550747871399
Epoch: 451
Train loss: 0.2682156264781952
Test loss: 0.2948809564113617
Epoch: 452
Train loss: 0.2681817412376404
Test loss: 0.2948072850704193
Epoch: 453
Train loss: 0.26814812421798706
Test loss: 0.29473409